Zuerst werden alle notwendigen Pakete und Module importiert.
Dabei wird neben den Modulen der Standardbibliothek (`random`, `sys`, `time` und `uuid`) und den Datenstrukturen `defaultdict` und `OrderedDict` aus dem `collections` Modul noch das Paket *Python-Chess* (`chess`) benötigt, welches bereits installiert wurde.
Zudem werden für die Anzeige einige Funktionen aus IPython benötigt, die durch die Installation und Verwendung von Jupyter Notebooks bereitgestellt werden.

In [ ]:
import random
import sys
import time
import uuid

import chess
import chess.engine
import chess.polyglot
import chess.svg

from collections import defaultdict
from collections import OrderedDict

from IPython.display import display, HTML, clear_output

Zunächst werden globale Konstanten definiert.

- `QUIESCENCE_SEARCH_DEPTH` ist die maximale Suchtiefe der Ruhesuche.
- `TABLE_SIZE` is die maximale größe der Transpositionstabellen.
- `TIMEOUT_SECONDS` wird verwendet, um die Zeit des Schachcomputers zur Berechnung seines nächsten Zuges festzulegen.

In [ ]:
QUIESCENCE_SEARCH_DEPTH: int = 20
TABLE_SIZE: int = 1.84e19
TIMEOUT_SECONDS: int = 30

Im folgenden Schritt werden einige globale Variablen definiert.
Dieser Schritt ist notwendig, um die Funktionsköpfe möglichst nah am Pseudocode aus dem Theorieteil und der Literatur zu halten.

- Die Variable `best_move` enthält den besten Zug für die aktuelle Iteration der `iterative_deepening`-Funktion.
- Demgegenüber speichert die Variable `global_best_move` den besten Zug aus allen Iterationen der iterativen Tiefensuche und somit den Zug, der vom Schachcomputer am Ende seines Zuges ausgeführt wird.
- `current_depth` speichert die aktuelle Tiefe für das Iterative Deepening.
- `endgame` ist eine Flag, welche auf True gesetzt wird, sobald sich das Spiel im Endspiel befindet.
- `is_timeout` ist genau dann wahr, wenn die Zeit des Schachcomputers abgelaufen ist.
- Bei `move_scores` handelt es sich um ein Dictionary, dass den Score für jeden bereits besuchten Zug im Iterative Deepening Verfahren speichert.
Dabei handelt es sich speziell um ein verschachteltes Dictionary.
Die Schlüssel auf der obersten Ebene bilden die Stellungen in \ac{FEN}.
Der dazugehörige Wert ist wiederum ein Dictionary mit den Zügen als Schlüssel und dem dazugehörigen Score als Wert.
- `piece_zobrist_values` ist eine Liste von Listen, wobei die inneren Listen jeweils ein Feld repräsentieren und ihrerseits für jede einzelne Figur einen eindeutigen Zobrist Hash beinhalten.
- In der Variablen `start_time` wird die Startzeit des Zuges des Schachcomputers abgelegt.
- `transposition_table` ist ein LRU Cache mit der maximalen Größe `TABLE_SIZE` und bildet damit die Transpositionstabellen ab.
- `zobrist_turn` ist ein Hash, welcher mit dem Zobrist Hash XOR verrechnet wird, falls die ziehende Person wechselt.

In [ ]:
best_move = None
current_depth = 0
endgame = False
global_best_move = None
is_timeout = False
move_scores = defaultdict(dict)
piece_zobrist_values = []
start_time = 0.0
transposition_table = None
zobrist_turn = 0

Nachdem alle notwendigen globalen Definitionen getätigt worden sind, wird nun die verwendete Bewertungsheuristik umgesetzt.
Da sich die Bewertungsfunktion in zwei Teile aufteilt, werden diese getrennt implementiert und am Ende zusammengefügt.
Zuerst erfolgt die Umsetzung der Figurenwerte.
Hierfür wird ein Dictionary `piece_values` angelegt, dass jeder Figur ihren entsprechenden Wert zuweist.

In [ ]:
piece_values = {
    chess.BISHOP: 330,
    chess.KING: 20_000,
    chess.KNIGHT: 320,
    chess.PAWN: 100,
    chess.QUEEN: 900,
    chess.ROOK: 500,
}

Die Funktion `get_piece_value` gibt den Figurenwert für eine übergebene Figur auf dem Schachbrett zurück.
Handelt es sich bei der Figur um eine der Farbe Schwarz, so wird der negierte Werte zurückgegeben, da der Schwarze Spieler der minimierende Spieler ist.

In [ ]:
def get_piece_value(piece: chess.Piece) -> int:
    factor = -1 if piece.color == chess.BLACK else 1
    return factor * piece_values.get(piece.piece_type)

Als nächstes werden die figurenspezifischen Positionstabellen implementiert.
Dafür wird ein Dictionary `piece_squared_tables` angelegt, das für jede Figur die entsprechende Positionstabelle speichert.
Die Tabellen werden dabei als Tupel von Tupeln gespeichert, da die Veränderung der Werte während des Spiels nicht zulässig ist.
Die Tabellen sind aus dem Theorieteil übernommen worden, weshalb sie aus Sicht des weißen Spielers zu betrachten sind.
Das Feld A1 befindet sich somit unten links, was dem Index `[7][0]` entspricht.

In [ ]:
piece_squared_tables = {
    chess.BISHOP: (
        (-20, -10, -10, -10, -10, -10, -10, -20),
        (-10,   0,   0,   0,   0,   0,   0, -10),
        (-10,   0,   5,  10,  10,   5,   0, -10),
        (-10,   5,   5,  10,  10,   5,   5, -10),
        (-10,   0,  10,  10,  10,  10,   0, -10),
        (-10,  10,  10,  10,  10,  10,  10, -10),
        (-10,   5,   0,   0,   0,   0,   5, -10),
        (-20, -10, -10, -10, -10, -10, -10, -20),
    ),
    chess.KING: (
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-20, -30, -30, -40, -40, -30, -30, -20),
        (-10, -20, -20, -20, -20, -20, -20, -10),
        ( 20,  20,   0,   0,   0,   0,  20,  20),
        ( 20,  30,  10,   0,   0,  10,  30,  20),
    ),
    chess.KNIGHT: (
        (-50, -40, -30, -30, -30, -30, -40, -50),
        (-40, -20,   0,   0,   0,   0, -20, -40),
        (-30,   0,  10,  15,  15,  10,   0, -30),
        (-30,   5,  15,  20,  20,  15,   5, -30),
        (-30,   0,  15,  20,  20,  15,   0, -30),
        (-30,   5,  10,  15,  15,  10,   5, -30),
        (-40, -20,   0,   5,   5,   0, -20, -40),
        (-50, -40, -30, -30, -30, -30, -40, -50),
    ),
    chess.PAWN: (
        (  0,   0,   0,   0,   0,   0,   0,   0),
        ( 50,  50,  50,  50,  50,  50,  50,  50),
        ( 10,  10,  20,  30,  30,  20,  10,  10),
        (  5,   5,  10,  25,  25,  10,   5,   5),
        (  0,   0,   0,  20,  20,   0,   0,   0),
        (  5,  -5, -10,   0,   0, -10,  -5,   5),
        (  5,  10,  10, -20, -20,  10,  10,   5),
        (  0,   0,   0,   0,   0,   0,   0,   0),
    ),
    chess.QUEEN: (
        (-20, -10, -10,  -5,  -5, -10, -10, -20),
        (-10,   0,   0,   0,   0,   0,   0, -10),
        (-10,   0,   5,   5,   5,   5,   0, -10),
        ( -5,   0,   5,   5,   5,   5,   0,  -5),
        (  0,   0,   5,   5,   5,   5,   0,  -5),
        (-10,   5,   5,   5,   5,   5,   0, -10),
        (-10,   0,   5,   0,   0,   0,   0, -10),
        (-20, -10, -10,  -5,  -5, -10, -10, -20),
    ),
    chess.ROOK: (
        (  0,   0,   0,   0,   0,   0,   0,   0),
        (  5,  10,  10,  10,  10,  10,  10,   5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        (  0,   0,   0,   5,   5,   0,   0,   0),
    ),
}

Im Endspiel wird für den König eine andere Tabelle verwendet. Diese ist nachfolgend definiert.

In [ ]:
kings_end_game_squared_table = (
    (-50, -40, -30, -20, -20, -30, -40, -50),
    (-30, -20, -10,   0,   0, -10, -20, -30),
    (-30, -10,  20,  30,  30,  20, -10, -30),
    (-30, -10,  30,  40,  40,  30, -10, -30),
    (-30, -10,  30,  40,  40,  30, -10, -30),
    (-30, -10,  20,  30,  30,  20, -10, -30),
    (-30, -30,   0,   0,   0,   0, -30, -30),
    (-50, -30, -30, -30, -30, -30, -30, -50),
)

Das Paket Python-Chess weist jeder Positionen einen Ganzzahlwert zu.
So erhält das Feld A1 den Zahlenwert 0, das Feld B1 den Wert 1.
Diese Zuweisung lässt sich bis zum letzten Feld H8 fortführen, das den Zahlenwert 63 erhält.
Um nachfolgend effizient auf die Positionswerte basierend auf dem Zahlenwert des Feldes zugreifen zu können, müssen die Zeilen der Positionstabellen umgekehrt werden, sodass Zeile 1 den Index 0 und die Zeile 8 den Index 7 erhält.
Dafür wird im Folgenden über jedes Schlüssel-Wert-Paar des `piece_squared_tables` Dictionary iteriert, die Tabelle in eine Liste konvertiert, diese Liste dann umgekehrt und anschließend in ein Tupel zurück überführt.
Analog wird die eine Tabelle, die in der Variablen `kings_end_game_squared_table` abgelegt ist, in eine Liste überführt und deren Elemente in umgekehrter Reihenfolge als Tupel in die Variable zurückgeschrieben.

In [ ]:
piece_squared_tables = {key: tuple(reversed(list(value))) 
                        for key, value in piece_squared_tables.items()}
kings_end_game_squared_table = tuple(reversed(list(kings_end_game_squared_table)))

Die bisher definierten Positionstabellen sind aus der Sicht des Weißen bzw. maximierenden Spielers definiert.
Da der zu entwickelnde Schachcomputer aber beide Farben und damit auch gegen sich selber spielen können soll, müssen noch die Positionstabellen für den minimierenden Spieler generiert werden.
Hierzu werden wie beim maximierenden Spieler die Zeilen der Tabellen umgekehrt.
Weil sich aber die Tabelle als ganzen um 180° drehen muss, werden zusätzlich alle Spalten, was den Elementen innerhalb der Zeilen entspricht, umgedreht.
Das resultierende Dictionary bzw. die resultierende Positionstabelle für den König in der Endphase, werden in den zugehörigen Variablennamen mit dem Präfix `reversed_` abgelegt.

In [ ]:
reversed_piece_squared_tables = {key: tuple([
                                            piece[::-1]
                                            for piece in value][::-1]) 
                                 for key, value in piece_squared_tables.items()}
reversed_kings_end_game_squared_table = tuple([
                                        piece[::-1]
                                        for piece in kings_end_game_squared_table][::-1])

Die Funktion `check_endgame` überprüft ob sich eine Stellung im Endspiel befindet. Dafür bekommt die Funktion eine Stellung (`board`) als Parameter. Anschließend wird über die Felder des Bretts iteriert und alle Figuren bis auf Bauern und Könige gezählt. Sobald der Zähler für die Figuren (`counter`) größer als 4 ist, ist sicher, dass sich das Spiel nicht im Endspiel befindet und es wird `False` zurückgegeben. Sollte der Zähler nach durchlaufen der Schleife immer noch kleiner als 4 sein, so befindet sich das Spiel im Endspiel und es wird `True` zurückgegeben.

In [ ]:
def check_endgame(board: chess.Board):
    counter = 0
    for square in range(64):
        piece = board.piece_at(square)
        if not piece:
            continue
        if piece.piece_type is not chess.PAWN and piece.piece_type is not chess.KING:
            counter += 1
            if counter > 4:
                return False
    return True

Die Funktion `get_piece_quared_tables_value` liefert den Wert der figurenspezifischen Positionstabellen für eine übergebene Figur `piece` auf dem Schachbrett.
Der Zeilen- und Spaltenindex in der Tabelle wird basierend auf dem Zahlenwert des Feldes, auf dem die Figur sich befindet (`square`), berechnet.
Dabei entspricht der Zeilenindex dem Ergebnis der Ganzzahldivision durch 8 und der Spaltenindex dem Rest aus der Division mit der Zahl 8.
Zudem wird ein Parameter `end_game` benötigt, der genau dann wahr ist, wenn sich das Spiel in der Endphase befindet und für den König eine abweichende Positionstabelle verwendet wird.
Des Weiteren wird erneut der gefundene Wert negiert, wenn es sich bei der Farbe um Schwarz und somit um den minimierenden Spieler handelt.

In [ ]:
def get_piece_squared_tables_value(piece: chess.Piece, square: int, end_game: bool = False) -> int:
    factor = -1 if piece.color == chess.BLACK else 1
    row = square // 8
    column = square % 8
    
    if end_game and piece.piece_type == chess.KING:
        if piece.color == chess.WHITE:
            return kings_end_game_squared_table[row][column]
        else:
            return reversed_kings_end_game_squared_table[row][column]
    
    if piece.color == chess.WHITE:
        piece_squared_table = piece_squared_tables.get(piece.piece_type)
    else:
        piece_squared_table = reversed_piece_squared_tables.get(piece.piece_type)
        
    return factor * piece_squared_table[row][column]

Die Funktion `simple_eval_heuristic` setzt die einfache Bewertungsheuristik um.
Sie erhält als Eingabeparameter die aktuelle Stellung `board` und ob sich das Spiel im Endspiel befindet (`end_game`).
Die Funktion schaut sich jedes Feld des Schachbretts an.
Befindet sich eine Figur auf dem Feld, so werden der Figurenwert und der Positionswert bestimmt und zum Endergebnis, das in der Variablen `piece_value` gespeichert wird, addiert.
Das Ergebnis wird dann zurückgegeben.

In [ ]:
def simple_eval_heuristic(board: chess.Board, end_game: bool = False) -> int:
    piece_value = 0
    for square in range(64):
        piece = board.piece_at(square)
        if not piece:
            continue
        piece_value += get_piece_value(piece)
        piece_value += get_piece_squared_tables_value(piece, square, end_game)
    return piece_value

Die beiden Dictionaries `zobrist_values_white`und `zobrist_values_black` werden für die Umsetzung des Zobrist Hashing benötigt.

In [ ]:
zobrist_values_white = {
    chess.PAWN: 1,
    chess.KNIGHT: 2,
    chess.BISHOP: 3,
    chess.ROOK: 4,
    chess.QUEEN: 5,
    chess.KING: 6,
}
zobrist_values_black = {
    chess.PAWN: 7,
    chess.KNIGHT: 8,
    chess.BISHOP: 9,
    chess.ROOK: 10,
    chess.QUEEN: 11,
    chess.KING: 12,
}

In der Funktion `intit_zobrist_list()` werden die Zobrist-Schlüssel für jede Figur auf jedem der 64 Schachfelder initialisiert.
Es wird `uuid` in Kombination mit dem Operatoren `& (1<<64)-1` verwendet, um einzigartige 64 Bit Hashes zu erzeugen.
Die erzeugten Hashes werden dann der Reihenfolge aus `zobrist_values_white` und `zobrist_values_black` in die zweidimensionale Liste `piece_zobrist_values` abgespeichert.
Anschließend wird die Funktion aufgerufen, um die Liste zu initialisieren.

In [ ]:
def init_zobrist_list():
    global piece_zobrist_values

    zobrist_turn = uuid.uuid4().int & (1 << 64) - 1
    for i in range(0, 64):
        NO_PIECE = uuid.uuid4().int & (1 << 64) - 1
        WHITE_PAWN = uuid.uuid4().int & (1 << 64) - 1
        WHITE_KNIGHT = uuid.uuid4().int & (1 << 64) - 1
        WHITE_BISHOP = uuid.uuid4().int & (1 << 64) - 1
        WHITE_ROOK = uuid.uuid4().int & (1 << 64) - 1
        WHITE_QUEEN = uuid.uuid4().int & (1 << 64) - 1
        WHITE_KING = uuid.uuid4().int & (1 << 64) - 1
        BLACK_PAWN = uuid.uuid4().int & (1 << 64) - 1
        BLACK_KNIGHT = uuid.uuid4().int & (1 << 64) - 1
        BLACK_BISHOP = uuid.uuid4().int & (1 << 64) - 1
        BLACK_ROOK = uuid.uuid4().int & (1 << 64) - 1
        BLACK_QUEEN = uuid.uuid4().int & (1 << 64) - 1
        BLACK_KING = uuid.uuid4().int & (1 << 64) - 1

        piece_zobrist_values.append(
            [
                NO_PIECE,
                WHITE_PAWN,
                WHITE_KNIGHT,
                WHITE_BISHOP,
                WHITE_ROOK,
                WHITE_QUEEN,
                WHITE_KING,
                BLACK_PAWN,
                BLACK_KNIGHT,
                BLACK_BISHOP,
                BLACK_ROOK,
                BLACK_QUEEN,
                BLACK_KING,
            ]
        )


init_zobrist_list()

`zobrist_hash()` wird verwendet, um eine Stellung in einen einzigartigen Zobrist-Schlüssel umzuwandeln.
Hierfür wird zunächst die zu überführende Stellung als Parameter (`board`) übergeben.
Anschließend wird ein leerer Hash erstellt (`zobrist_hash = 0`), welcher dann den Figuren und Feldern der Stellung entsprechend mit den spezifischen Zobrist-Schlüsseln aus der Liste `piece_zobrist_values` XOR verrechnet wird.
Um auf die richtigen Indizes der Figurtypen zuzugreifen, werden die Dictionaries `zobrist_value_white` und `zobrist_value_black` verwendet.

In [ ]:
def zobrist_hash(board: chess.Board) -> int:
    global zobrist_turn

    zobrist_hash = 0
    if board.turn == chess.WHITE:
        zobrist_hash = zobrist_hash ^ zobrist_turn

    for square in range(64):
        piece = board.piece_at(square)
        if not piece:
            index = 0
        elif piece.color == chess.WHITE:
            index = zobrist_values_white.get(piece.piece_type)
        elif piece.color == chess.BLACK:
            index = zobrist_values_black.get(piece.piece_type)
        zobrist_hash = zobrist_hash ^ piece_zobrist_values[square][index]

    return zobrist_hash

`zobrist_move()` ändert einen bestehenden Zobrist-Schlüssel (`zobrist_hash`), der die Stellung (`board`) repräsentiert, nachdem ein Zug (`move`) ausgeführt wurde.

- Als erstes wird bestimmt, welcher Spieler am Zug ist, weil diese Information benötigt wird, um zwischen den Dictionaries `zobrist_value_white` und `zobrist_value_black` unterscheiden zu können.
- Danach werden die beiden Felder bestimmt, von wo nach wo die ziehende Figur sich bewegt.
- Anschließend werden die Figurtypen auf den entsprechenden Feldern bestimmt.
Falls sich keine Figur auf dem Feld befindet auf das die ziehende Figur hinzieht, so wird `NO_PIECE` verwendet, welcher sich im Array `piece_zobrist_values`  immer auf den Positionen `[X][0]` befindet (X in range(0, 64)).
- Um den Zug auszuführen wird die Folgende Reihenfolge von XOR Operationen durchgeführt: Zuerst wird der Figurtyp auf dem Startfeld mit sich selbst verrechnet. Anschließend wird `NO_PIECE` auf diesem Feld verrechnet, weil das Feld nachdem die ziehende Figur wegzieht, von keiner Figur besetzt ist. Danach wird der Figurentyp auf dem Zielfeld mit sich selbst verrechnet. Zuletzt wird der ziehende Figurtyp auf dem Zielfeld verrechnet.
- Am Schluss wird noch die Flag `zobrist_turn` verrechnet, weil nach jedem Zug der ziehende Spieler wechselt. 


In [ ]:
def zobrist_move(board,move,zobrist_hash):
    white_to_move = board.turn
    from_square = move.from_square
    to_square = move.to_square
    moving_piece = board.piece_at(from_square)
    captured_piece = board.piece_at(to_square)

    if white_to_move:
        zobrist_hash = zobrist_hash ^ piece_zobrist_values[from_square][zobrist_values_white.get(moving_piece.piece_type)]
        zobrist_hash = zobrist_hash ^ piece_zobrist_values[from_square][0]
        if not captured_piece:
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][0]
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][zobrist_values_white.get(moving_piece.piece_type)]
        else:
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][zobrist_values_black.get(captured_piece.piece_type)]
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][zobrist_values_white.get(moving_piece.piece_type)]
    else:
        zobrist_hash = zobrist_hash ^ piece_zobrist_values[from_square][zobrist_values_black.get(moving_piece.piece_type)]
        zobrist_hash = zobrist_hash ^ piece_zobrist_values[from_square][0]
        if not captured_piece:
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][0]
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][zobrist_values_black.get(moving_piece.piece_type)]
        else:
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][zobrist_values_white.get(captured_piece.piece_type)]
            zobrist_hash = zobrist_hash ^ piece_zobrist_values[to_square][zobrist_values_black.get(moving_piece.piece_type)]
            
    zobrist_hash = zobrist_hash^zobrist_turn
    return zobrist_hash

Für die spätere Implementierung der Transpositionstabellen wird ein LRU Cache implementiert repräsentiert durch die Klasse `LRUCache`.
Eine detaillierte Beschreibung der Implementierung ist in Kapitel~\ref{ch:transpositional-tables} zu finden.

In [ ]:
class LRUCache:
    def __init__(self, size):
        self.od = OrderedDict()
        self.size = size

    def get(self, key, default=None):
        try:
            self.od.move_to_end(key)
        except KeyError:
            return default
        return self.od[key]

    def __contains__(self, item):
        return item in self.od

    def __len__(self):
        return len(self.od)

    def __getitem__(self, key):
        self.od.move_to_end(key)
        return self.od[key]

    def __setitem__(self, key, value):
        try:
            del self.od[key]
        except KeyError:
            if len(self.od) == self.size:
                self.od.popitem(last=False)
        self.od[key] = value


transposition_table = LRUCache(TABLE_SIZE)

Die Funktion `quiescence_search_maximize` implementiert die Ruhesuche aus der Sicht des maximierenden Spielers. Sie ist daher weitesgehend identisch mit der Funktion maximize und verfügt auch über die selbe Parameterliste. Ist die maximale Tiefe für die Ruhesuche (`QUIESCENCE_SEARCH_DEPTH`) erreicht, so wird die aktuelle Stellung mittels der Bewertungsheuristik evaluiert und das Ergebnis zurückgeliefert.
Bei den Zügen werden nur die, die als favorisierend betrachtet werden (`favorable_moves`), berücksichtigt.

In [ ]:
def quiescence_search_maximize(board: chess.Board, alpha, beta, currentDepth: int):
    global best_move
    global global_best_move
    global endgame

    if currentDepth == QUIESCENCE_SEARCH_DEPTH:
        return simple_eval_heuristic(board, endgame)

    favorable_moves = []
    moves = board.legal_moves

    for move in moves:
        if is_favorable_move(board, move):
            favorable_moves.append(move)

    if favorable_moves == []:
        return simple_eval_heuristic(board)

    score = alpha
    for move in favorable_moves:
        board.push(move)
        move_score = quiescence_search_minimize(board, score, beta, currentDepth + 1)
        board.pop()
        if move_score > score:
            score = move_score
            if score >= beta:
                break

    return score

Analog wird die Funktion für die minimierende Ruhesuche `quiescence_search_min` implementiert.

In [ ]:
def quiescence_search_minimize(board: chess.Board, alpha, beta, currentDepth: int):
    global best_move
    global global_best_move
    global endgame

    if currentDepth == QUIESCENCE_SEARCH_DEPTH:
        return simple_eval_heuristic(board, endgame)

    moves = board.legal_moves
    favorable_moves = []

    for move in moves:
        if is_favorable_move(board, move):
            favorable_moves.append(move)

    if favorable_moves == []:
        return simple_eval_heuristic(board)

    score = beta
    for move in favorable_moves:
        board.push(move)
        move_score = quiescence_search_maximize(board, alpha, score, currentDepth + 1)
        board.pop()
        if move_score < score:
            score = move_score
            if score <= alpha:
                break

    return score

Die Funktion `is_favorable_move` überprüft, ob ein Zug zu einer, wie im Kapitel Ruhesuche beschrieben, vorteilhaften Stellung führt oder nicht. 
Als Argumente bekommt die Funktion eine Stellung (`board`) und einen Zug (`move`) übergeben.
Nun wird überprüft, ob der Zug vorteilhaft ist.
Der Zug ist dann vorteilhaft, wenn der Zug ein Schlagzug ist und wenn die schlagende Figur weniger Wert ist als die Figur, welche geschlagen wird oder wenn das Feld von der schlagenden Seite feldbeherrschungstechnisch majorisiert wird.
En-Passant Züge werden zuvor schon herausgefiltert, weil in diesem Fall ein Bauer geschlagen wird, welcher sich aktuell auf einem anderen Feld befindet, als das Zielfeld des Schlagzugs. 

In [ ]:
def is_favorable_move(board: chess.Board, move: chess.Move) -> bool:
    if move.promotion is not None:
        return True
    if board.is_capture(move) and not board.is_en_passant(move):
        if piece_values.get(board.piece_type_at(move.from_square)) < piece_values.get(
            board.piece_type_at(move.to_square)
        ) or len(board.attackers(board.turn, move.to_square)) > len(
            board.attackers(not board.turn, move.to_square)
        ):
            return True
    return False

Nachdem die verwendete Bewertungsheuristik implementiert ist, kann nun das Alpha-Beta Pruning zusammen mit dem Iterative Deepening umgesetzt werden.
Hierfür wird eine Funktion `iterative_deepening` implementiert, die den Ablauf dieses Prozesses koordiniert.
Sie erhält folgende drei Parameter:

- Der Parameter `board` enthält die aktuelle Stellung.
- Die initiale Tiefe, bei der gesucht wird, wird mit dem Parameter `depth` übergeben.
- `color` repräsentiert den Spieler, der aktuell am Zug ist.

Als erstes wird überprüft, ob sich das Spiel aktuell im Endspiel befindet und falls ja, wird die globale Flag `endgame` auf `True` gesetzt.
Alle anderen benötigten Informationen werden global abgelegt.
Zu Beginn des Iterative Deepening wird geprüft, ob der Spieler nur einen erlaubten Zug machen kann.
Ist dies der Fall, so wird er sofort zurückgegeben und keine weiteren Berechnungen sind von Nöten.
Ist mehr als ein zulässiger Zug möglich, so wird die aktuelle Systemzeit global gespeichert und die Variable `d` mit 0 initialisiert.
`d` repräsentiert hierbei die Tiefe, die zur initialen Tiefe `depth` hinzugefügt wird.
In einer Endlosschleife wird sukzessiv die Suchtiefe erhöht und `minimize` bzw. `maximize` aufgerufen.
Sowohl `alpha` als auch `beta` werden mit einem sehr hohen bzw. sehr niedrigen Zahlenwert initialisiert.
Bei jedem Schleifendurchlauf wird dabei dem globale besten Zug (`global_best_move`) der Zug aus der Variablen `best_move` zugewiesen.
Falls der Return Wert `sys.maxsize` oder `-sys.maxsize` entspricht, bedeutet dies, dass ein Matt von einem der beiden Spieler erzwungen werden kann und deshalb wird die Suche hier abgebrochen.
Des Weiteren wird überprüft, ob die Zeit des Schachcomputers abgelaufen ist.
Ist dies der Fall, wird der globale beste Zug zurückgegeben.

In [ ]:
def iterative_deepening(board: chess.Board, depth: int, color: int):
    global best_move
    global current_depth
    global global_best_move
    global is_timeout
    global start_time
    global endgame

    if not endgame and check_endgame(board):
        endgame = True

    zobrist = zobrist_hash(board)
    if board.legal_moves == 1:
        return board.legal_moves[0]

    is_timeout = False
    start_time = time.time()
    d = 0
    current_score = 0

    while True:
        if d > 1:
            global_best_move = best_move
            print(f"Completed search with depth {current_depth}. Best move so far: {global_best_move} (Score: {current_score})")
        if current_score == sys.maxsize or current_score == -sys.maxsize:
            return global_best_move
        current_depth = depth + d
        if color == chess.BLACK:
            current_score = minimize(board, current_depth, -sys.maxsize, sys.maxsize, color, zobrist)
        else:
            current_score = maximize(board, current_depth, -sys.maxsize, sys.maxsize, color, zobrist)
        d += 1
        if is_timeout:
            return global_best_move

Die Implementierung der Funktion `maximize` orientiert sich stark am Pseudocode aus dem Theorieteil.
Die Funktion erhält fünf Parameter.

- `board` enthält die aktuelle Stellung.
- `depth` repräsentiert die Tiefe, mit der `maximize` aufgerufen wird.
- `alpha` repräsentiert die Variable Alpha des Alpha-Beta Prunings.
- `beta` repräsentiert die Variable Beta des Alpha-Beta Prunings.
- `zobrist` enthält den Zobrist Hash der aktuellen Stellung

Zu Beginn wird geprüft, ob die Zeit des Schachcomputers bereits abgelaufen ist.
Ist dies der Fall, so wird die Variable `is_timeout` auf `True` gesetzt und `alpha` zurückgegeben.
Anschließend wird geprüft ob sich die Stellung in einem Schachmatt `board.is_checkmate()` oder einem Patt `board.is_stalemate` befindet, oder ob sich ein Unentschieden über die dreifache Wiederholung einer Stellung erzwingen lässt. Falls einer dieser drei Fälle zutrifft, wird sofort der entsprechende Score (`sys.maxsiz` bzw. `-sys.maxsize` und `0`) zurückgegeben
Danach wird überprüft, ob  für den `zobrist_hash` der aktuellen Stellung und der aktuellen Tiefe `depth` ein Eintrag in der Transpositionstabelle `transposition_table` vorhanden ist. Ist dies der Fall, so werden anschließend die aktuellen Alpha und Beta Werte mit den Alpha und Beta Werten aus der Transpositionstabelle verglichen.
Falls sich das aktuelle Alpha-Beta Intervall innerhalb des Alpha-Beta Intervalls der Transpositionstabelle befindet, wird einfach der `score` der Transpositionstabelle als return Wert wiedergegeben. Sonst wird das Minimum von beiden Alpha-Werten und das Maximum von beiden Beta Werten bestimmt und diese als aktuelle Alpha und Beta Werte übernommen.
Anschließend wird überprüft, ob es sich bei der aktuellen Suchtiefe um eine Tiefe kleiner als 1 handelt.
Ist dies der Fall, wird die aktuelle Stellung mittels der Ruhesuche evaluiert und der entsprechende Wert zurückgeliefert.
Weil noch kein Zug durchgeführt wurde, wird die Ruhesuche für `maximize` aufgerufen (`quiescnece_search_maximize`).
Dabei ist zu beachten, dass die Suchtiefe um eins erhöht wird.
Ist die aktuelle Suchtiefe nicht kleiner als eins, so wird als nächstes der Variablen `score` der Wert für `alpha` zugewiesen.
Anschließend werden alle zulässigen Züge nach ihren Scores sortiert, sodass aussichtsreichere Züge zuerst evaluiert werden.
Ist kein Score für einen zulässigen Zug vorhanden, so werden die Züge mit einem neutralen Score von 0 belegt.
Nachfolgend wird über alle zulässigen Züge iteriert und jeweils die Zobrist Hashes für die neu entstehenden Stellungen mit `zobrist_move` berechnet und den folgenden Aufrufen der `minimize` Funktion als Parameter übergeben.
Die Variablen `score` und `alpha` werden basierend auf dem Wert von `move_score` aktualisiert, wie dies im Pseudocode aus dem Theorieteil der Fall ist.
Zusätzlich wird `best_move` der aktuelle Zug zugewiesen, wenn folgende zwei Bedingungen erfüllt sind:

1. Der aktuelle Score ist größer als Alpha, wodurch die Variable `alpha` aktualisiert wird und ein möglicher neuer bester Zug gefunden wurde.
1. Die aktuelle Suchtiefe entspricht der Suchtiefe, mit der die Funktion `iterative_deepening` die aktuelle Iteration anstieß.

Bevor letztlich der Score der aktuellen Stellung zurückgeliefert wird, wird Stellung und Score mit den aktuellen Alpha und Beta Werten in der Transpositionstabelle abgespeichert. 
Während des Schleifendurchlaufs wird der `move_score` für einen Zug im Dictionary `move_scores` abgelegt.
Dabei wird er der Stellung zugewiesen, die als Ausgangsstellung für den Zug dient.

In [ ]:
def maximize(board: chess.Board, depth: int, alpha: int, beta: int, color: int, zobrist: int) -> int:
    global is_timeout
    global start_time
    global best_move
    global global_best_move

    if time.time() - start_time > TIMEOUT_SECONDS:
        is_timeout = True
        return alpha

    if board.is_checkmate():
        return -sys.maxsize
    if board.is_stalemate():
        return 0
    if board.can_claim_threefold_repetition():
        return 0

    if (zobrist, depth) in transposition_table:
        score, a, b = transposition_table[(zobrist, depth)]
        if a <= alpha and beta <= b:
            return score
        else:
            alpha = min(alpha, a)
            beta = max(beta, b)

    if depth < 1:
        return quiescence_search_maximize(board, alpha, beta, 1)

    score = alpha

    board_scores = move_scores.get(board.fen(), dict())
    moves = sorted(
        board.legal_moves, key=lambda move: -board_scores.get(move, 0),
    )

    for move in moves:
        new_zobrist = zobrist_move(board, move, zobrist)
        board.push(move)
        move_score = minimize(board, depth - 1, score, beta, color, new_zobrist)
        move_scores[board.fen()][move] = move_score
        board.pop()

        if move_score > score:
            score = move_score
            if depth == current_depth:
                best_move = move
            if score >= beta:
                break

    transposition_table[(zobrist, depth)] = score, alpha, beta
    return score

Die Funktion `minimize` ist zu großen Teilen mit der Funktion `maximize` identisch, wie dies auch im Pseudocode aus dem Theorieteil der Fall ist.
Die Unterschiede zur `maximize`-Implementierung sind:

- Die Liste der zulässigen Züge ist aufwärts und nicht abwärts sortiert.
Somit fängt der Schachcomputer mit dem Zug an, der den geringsten Score besitzt.
Zudem ist der für den Spieler schlechteste Zug nun `sys.maxsize`.
- Es wird die Variable `beta` aktualisiert und nicht die Variable `alpha`.
Diese wird zudem aktualisiert, wenn der aktuelle `score` niedriger und somit besser ist.

In [ ]:
def minimize(board: chess.Board, depth: int, alpha: int, beta: int, color: int, zobrist) -> int:
    global best_move
    global global_best_move
    global is_timeout
    global start_time
    
    if time.time() - start_time > TIMEOUT_SECONDS:
        is_timeout = True
        return beta
    
    if board.is_checkmate():
        return sys.maxsize
    if board.is_stalemate():
        return 0
    if board.can_claim_threefold_repetition():
        return 0
    
    if (zobrist, depth) in transposition_table:
        score, a, b = transposition_table[(zobrist, depth)]
        if a <= alpha and beta <= b:
            return score
        else:
            alpha = min(alpha, a)
            beta = max(beta , b)
    
    if depth < 1:
        return quiescence_search_minimize(board, alpha, beta, 1)
    
    score = beta
    
    board_scores = move_scores.get(board.fen(), dict())
    moves = sorted(
        board.legal_moves, key=lambda move: board_scores.get(move, 0),
    )

    for move in moves:
        new_zobrist = zobrist_move(board, move, zobrist)
        board.push(move)
        move_score = maximize(board, depth - 1, alpha, score, color, new_zobrist)
        move_scores[board.fen()][move] = move_score
        board.pop()

        if move_score < score:
            score = move_score
            if depth==current_depth:
                best_move = move
            if score <= alpha:
                break

    transposition_table[(zobrist, depth)] = score, alpha, beta
    return score

Die Funktion `get_opening_data_base_moves` ist eine Hilfsfunktion, die einen Zug aus der Opening Data Base für die aktuelle Stellung (`board`) zurückgibt, falls ein Zug für die übergebene Stellung gefunden wird.

In [ ]:
def get_opening_data_base_moves(board: chess.Board):
    move = None
    opening_moves = []

    with chess.polyglot.open_reader("Performance.bin") as reader:
        for entry in reader.find_all(board):
            opening_moves.append(entry)

    if opening_moves:
        random_entry = random.choice(opening_moves)
        move = random_entry.move
        print(move)

    return move

Für die Anzeige werden die Spielernamen benötigt.
Hierfür wird eine Hilfsfunktion `who` implementiert, die einen Spieler als Parameter erhält und den zugehörigen Namen als Zeichenkette zurückgibt.

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

Nachdem der Schachcomputer implementiert ist, wird nun die Schnittstelle für den menschlichen Spieler implementiert.
Hierfür wird zuerst eine Hilfsfunktion `get_move` benötigt, die die Nutzereingabe in einen Zug umwandelt, sofern dies möglich ist.
Zudem wird in ihr überprüft, ob das Spiel vorzeitig zu beenden ist.
Dies ist der Fall, wenn der Nutzer als Zug ein `q` (für engl. *quit*) eingibt.
Sie erhält als Parameter den dem Nutzer anzuzeigenen Text für die Zugeingabe.

In [ ]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()

    try:
        chess.Move.from_uci(uci)
    except:
        uci = None

    return uci

Die Funktion `human_player` repräsentiert den menschlichen Spieler und koordiniert die Züge des Spielers.
Dafür wird zuerst die aktuelle Stellung mittels der IPython-Funktion `display` angezeigt.
Anschließend werden dem menschlichen Spieler alle zulässigen Züge für die aktuelle Stellung angezeigt und er wird aufgefordert, seinen Zug zu tätigen.
Diese Aufforderung geschieht solange, bis der Nutzer einen gültigen Zug eingegeben hat oder aber das Abbruchkriterium `q`.

In [ ]:
def human_player(board):
    display(board)
    uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    legal_uci_moves = [move.uci() for move in board.legal_moves]

    while uci not in legal_uci_moves:
        print(f"Legal moves: {(', '.join(sorted(legal_uci_moves)))}")
        uci = get_move(f"{who(board.turn)}'s move [q to quit]>")

    return uci

Nachdem alle notwendigen Funktionen für den Schachcomputer implementiert sind, kann nun die Funktion `ai_player` implementiert werden.
Sie repräsentiert den Schachcomputer dem Spiel gegenüber.
Die Funktion besitzt zwei Parameter:

- `board` enthält die aktuelle Stellung.
- `color` repräsentiert den Spieler, der am Zug ist.

Zuerst wird in der verfügbaren Opening Data Base geschaut, ob ein Zug für die übergebene Stellung vorhanden ist.
Ist dies der Fall, so wird der Zug zurückgegeben.
Andernfalls wird das globale `move_scores` Dictionary zurückgesetzt und die Funktion `iterative_deepening` aufgerufen und deren gewählter Zug zurückgegeben.
Es ist zu beachten, dass wenn ein Zug aus der Opening Data Base verwendet wird, die globalen Dictionaries für die Zugscores und die Nachbarstellungen zurückgesetzt werden, da ansonsten keine ausreichende Kontrolle über die Tiefen existiert.
Diese Gegebenheit kann durch eine komplexere Implementierung der Zugsortieren optimiert werden, wird an dieser Stellung aber nicht weiter betrachtet.

In [ ]:
def ai_player(board: chess.Board, color: int):
    move = get_opening_data_base_moves(board)
    
    if move:
        moves_scores = defaultdict(dict)
        return move
    else:
        return iterative_deepening(board, 0, color)

`play_game` ist die Funktion, die den Spielablauf koordiniert.
Der Funktion kann optional ein Parameter `pause` übergeben werden.
Dieser legt fest, wie lange die Stellung und der zu ihr geführte Zug angezeigt werde soll, bevor der andere Spieler am Zug ist.
Der übergebene Zahlenwert wird als Anzahl an Sekunden interpretiert.
Nach der Initialisierung des Schachbretts sind solange beide Spieler abwechseld am Zug, bis das Spiel als beendet angesehen wird oder das Spiel abgebrochen wurde.
Anschließend werden je nach Ausgang des Spiels unterschiedliche Ergebnisse angezeigt.

In [ ]:
def play_game(pause=1):
    board = chess.Board()
    
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                #move = board.parse_uci(human_player(board))
                move = ai_player(board, chess.WHITE)
            else:
                move = ai_player(board, chess.BLACK)
                
            name = who(board.turn)
            board.push(move)
            html = "<br/>%s </b>"%(board._repr_svg_())
            clear_output(wait=True)
            display(HTML(html))
            time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted"
        return None, msg, board
    
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: fivefold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    
    print(msg)
    return result, msg, board

In [ ]:
play_game()